In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-04 04:18:38,701 INFO: Initializing external client
2025-03-04 04:18:38,701 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 04:18:39,937 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Fetching data from 2025-02-03 09:18:38.701349+00:00 to 2025-03-04 08:18:38.701349+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.19s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-03 10:00:00+00:00,2,0
1,2025-02-03 11:00:00+00:00,2,0
2,2025-02-03 12:00:00+00:00,2,0
3,2025-02-03 13:00:00+00:00,2,0
4,2025-02-03 14:00:00+00:00,2,0
...,...,...,...
174189,2025-03-04 03:00:00+00:00,263,3
174190,2025-03-04 04:00:00+00:00,263,8
174191,2025-03-04 05:00:00+00:00,263,18
174192,2025-03-04 06:00:00+00:00,263,89


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174194 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174194 non-null  int32                  
 2   rides               174194 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174194 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174194 non-null  datetime64[us]
 1   pickup_location_id  174194 non-null  int32         
 2   rides               174194 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,32,48,92,76,8,155,62,60,6,80,...,14,13,5,21,120,218,146,111,263,2025-03-02 00:00:00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,210,2025-02-26 19:00:00
2,3,0,0,0,0,2,0,2,0,1,...,0,2,1,2,1,1,1,1,216,2025-02-14 06:00:00
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,147,2025-02-14 01:00:00
4,3,2,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,217,2025-02-04 16:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,67,2025-02-15 05:00:00
247,0,0,0,0,0,0,4,0,1,0,...,0,1,0,0,0,1,1,0,86,2025-02-26 15:00:00
248,0,0,1,0,0,0,2,2,0,1,...,0,1,3,0,0,0,0,2,62,2025-02-28 07:00:00
249,243,130,148,572,217,231,173,141,160,167,...,0,133,311,225,168,179,37,2,142,2025-02-13 20:00:00


In [12]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-04 04:21:39,718 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 04:21:39,723 INFO: Initializing external client
2025-03-04 04:21:39,726 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 04:21:40,365 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697
Fetching data from 2025-02-03 09:21:39.718008+00:00 to 2025-03-04 08:21:39.718008+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.53s) 


In [13]:
current_date

Timestamp('2025-03-04 09:21:39.718008+0000', tz='Etc/UTC')

In [14]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-03 10:00:00
1,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,1,1,1,0,3,2025-03-03 10:00:00
2,6,2,2,2,4,4,1,4,0,1,...,1,2,0,1,1,7,11,4,4,2025-03-03 10:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,6,2025-03-03 10:00:00
4,1,2,4,2,2,4,1,1,1,5,...,0,1,1,1,2,2,5,3,7,2025-03-03 10:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,1,0,1,0,0,0,0,0,...,0,0,0,0,2,0,2,0,259,2025-03-03 10:00:00
247,0,0,5,2,4,3,3,1,1,3,...,0,0,0,1,5,0,3,0,260,2025-03-03 10:00:00
248,7,26,13,25,37,25,38,30,41,24,...,0,0,2,2,2,2,9,12,261,2025-03-03 10:00:00
249,98,76,87,70,71,102,81,98,73,43,...,1,2,9,16,69,139,172,124,262,2025-03-03 10:00:00


In [15]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-04 04:22:40,568 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-04 04:22:40,578 INFO: Initializing external client
2025-03-04 04:22:40,580 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-04 04:22:41,644 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214697

In [16]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [17]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,3.0
3,6,0.0
4,7,2.0
...,...,...
246,259,0.0
247,260,0.0
248,261,14.0
249,262,110.0


In [18]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 237, 138, 186, 161, 162, 239, 142, 170, 230])